# Machine Learning

## Machine Learning Decision Tree

This application uses a "wireless indoor localization dataset".  Essentially it represents wi-fi signals (1-7) and a room for each.  The algorithm will decide based on signal, which room the device lives in. 

The decision tree uses "Gini impurity" and the training algorithm is a recursive algorithm called CART, short for Classification And Regression Trees.

The code is broken up by:

* Example of the dataset
* A class to define the nodes
* A class for the CART algorith
* An example test case

At the end of the program, will atempt to predict and test results.  The information is printed to the screen.  By passing in an example record representing strength, the application will determine which room the device is in.

In [60]:
#===================================================================
#   Date: 10/16/2021
#   Description - Decision Tree by Hand
#===================================================================

import numpy as np

#### Following is an example of the data

It gives 7 features representing the strength of 7 Wi-Fi signals perceived by a phone in an apartment, along with the indoor location of the phone which can be Room 1, 2, 3 or 4

In [61]:
# Example rows of the data
# Show first few rows
data = pd.read_csv('wifi_localization.txt')
data.head()

,-64\t-56\t-61\t-66\t-71\t-82\t-81\t1
0,-68\t-57\t-61\t-65\t-71\t-85\t-85\t1
1,-63\t-60\t-60\t-67\t-76\t-85\t-84\t1
2,-61\t-60\t-68\t-62\t-77\t-90\t-80\t1
3,-63\t-65\t-60\t-63\t-77\t-81\t-87\t1
4,-64\t-55\t-63\t-66\t-76\t-88\t-83\t1


#### Binary tree with decision tree semantics

Following is the binary tree node that will be used for this decision tree project.

In [62]:
# Node class, this is used by the decision tree itself
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

#### Decision tree using the CART algorithm

Following is the decision tree for this project.  It uses the CART algorithm.

In [63]:
# Decision tree algorithm
# Uses the above node class
class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    # Function to predict results based on the input
    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

#### Decision tree test

The following code tests the decision tree.  It passes the signal record "0, 0, 5, 1.5" and detects which room the device is in.  In this case, room 1.

In [64]:
# Main function that executes the test
# Uses the DeisionTreeClassifier class from above

if __name__ == "__main__":
    import sys
    from sklearn.datasets import load_iris

    dataset = load_iris()
    X, y = dataset.data, dataset.target
    clf = DecisionTreeClassifier(max_depth=1)
    clf.fit(X, y)
    # Test and print results
    print(clf.predict([[0, 0, 5, 1.5]]))

[1]
